In [ ]:
!pip install psycopg2

In [ ]:
!pip install py7zr

In [3]:
import psycopg2
import sqlalchemy

import pandas as pd

import time

from ftplib import FTP
import os
import re
import py7zr

import glob

In [3]:
import sys
print(sys.version.split()[0])

3.9.7


In [3]:
#Informações de conexão com o banco Postgres
host='dataiesb.iesbtech.com.br'
dbname='IESB_Empresas'
user='1922120026_TCC_Thiago'
password='***********'

In [4]:
# Função para criar conexão no banco
def conecta_db():
    con = psycopg2.connect(host=host, 
                         database=dbname,
                         user=user, 
                         password=password)
    return con

def postgres_test():
    try:
        con = conecta_db()
        con.close()
        print("Conexão com o banco de dados realizada com sucesso!")
    except:
        print("A conexão com o banco de dados falhou!")

postgres_test()

Conexão com o banco de dados realizada com sucesso!


In [5]:
# Função para criar tabela no banco
def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()

In [6]:
# Criando a tabela para os dados das CAGED EXCLUIDOS
sql = '''CREATE TABLE IF NOT EXISTS tb_caged_exc (
competenciamov varchar(255),
regiao varchar(255),
uf varchar(255),
municipio varchar(255),
secao varchar(255),
subclasse varchar(255),
saldomovimentacao varchar(255),
cbo2002ocupacao varchar(255),
categoria varchar(255),
graudeinstrucao varchar(255),
idade varchar(255),
horascontratuais varchar(255),
racacor varchar(255),
sexo varchar(255),
tipoempregador varchar(255),
tipoestabelecimento varchar(255),
tipomovimentacao varchar(255),
tipodedeficiencia varchar(255),
indtrabintermitente varchar(255),
indtrabparcial varchar(255),
salario varchar(255),
tamestabjan varchar(255),
indicadoraprendiz varchar(255),
origemdainformacao varchar(255),
competenciadec varchar(255),
competenciaexc varchar(255),
indicadordeexclusao varchar(255),
indicadordeforadoprazo varchar(255),
unidadesalariocodigo varchar(255),
valorsalariofixo varchar(255)
  );'''
criar_db(sql)

# Criando a tabela para os dados das CAGED MOVIMENTOS
sql = '''CREATE TABLE IF NOT EXISTS tb_caged_mov (
competenciamov varchar(255),
regiao varchar(255),
uf varchar(255),
municipio varchar(255),
secao varchar(255),
subclasse varchar(255),
saldomovimentacao varchar(255),
cbo2002ocupacao varchar(255),
categoria varchar(255),
graudeinstrucao varchar(255),
idade varchar(255),
horascontratuais varchar(255),
racacor varchar(255),
sexo varchar(255),
tipoempregador varchar(255),
tipoestabelecimento varchar(255),
tipomovimentacao varchar(255),
tipodedeficiencia varchar(255),
indtrabintermitente varchar(255),
indtrabparcial varchar(255),
salario varchar(255),
tamestabjan varchar(255),
indicadoraprendiz varchar(255),
origemdainformacao varchar(255),
competenciadec varchar(255),
indicadordeforadoprazo varchar(255),
unidadesalariocodigo varchar(255),
valorsalariofixo varchar(255)
  );'''
criar_db(sql)

# Criando a tabela para os dados das CAGED FORA DO PRAZO
sql = '''CREATE TABLE IF NOT EXISTS tb_caged_for (
competenciamov varchar(255),
regiao varchar(255),
uf varchar(255),
municipio varchar(255),
secao varchar(255),
subclasse varchar(255),
saldomovimentacao varchar(255),
cbo2002ocupacao varchar(255),
categoria varchar(255),
graudeinstrucao varchar(255),
idade varchar(255),
horascontratuais varchar(255),
racacor varchar(255),
sexo varchar(255),
tipoempregador varchar(255),
tipoestabelecimento varchar(255),
tipomovimentacao varchar(255),
tipodedeficiencia varchar(255),
indtrabintermitente varchar(255),
indtrabparcial varchar(255),
salario varchar(255),
tamestabjan varchar(255),
indicadoraprendiz varchar(255),
origemdainformacao varchar(255),
competenciadec varchar(255),
indicadordeforadoprazo varchar(255),
unidadesalariocodigo varchar(255),
valorsalariofixo varchar(255)
  );'''
criar_db(sql)

In [6]:
def load_caged_to_db(table_name, colnames, origem, ano, mes):
    
    # verifica se o arquivo existe
    if os.path.isfile(f'{local_dir}/{origem}{ano}{mes}.txt'):
        # Leitura do arquivo txt e salvando como csv
        df_txt = pd.read_table(f'{local_dir}/{origem}{ano}{mes}.txt', delimiter=";", encoding='utf-8', quotechar='"')
        df_txt.to_csv(f'{local_dir}/{origem}{ano}{mes}.csv', sep=';', index=False, quotechar='"')

        # Conexão com o banco de dados
        engine = sqlalchemy.create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')
        engine = engine.connect()

        # Leitura em lotes do arquivo csv e gravação na tabela do banco de dados
        for df in pd.read_csv(f'{local_dir}/{origem}{ano}{mes}.csv',
                              delimiter=";",
                              header=0,
                              chunksize=65000,
                              names=colnames,
                              iterator=True,
                              dtype=str,
                              encoding="utf-8",
                              quotechar='"'):
            # Usando Try para tentativas de conexão, caso perca a conexão espera 60 segundos para tentar novamente o envio
            try:
                print("Iniciando inclusão de dados no banco...")
                df.to_sql(table_name, engine, if_exists="append", index=False, method="multi")
            except:
                time.sleep(60)
                print("Iniciando inclusão de dados no banco...")
                df.to_sql(table_name, engine, if_exists="append", index=False, method="multi")

        # Fechando conexão com o banco de dados
        engine.close()

In [7]:
# Tabelas CAGED EXCLUÍDOS
colnames1=[
    'competenciamov',
    'regiao',
    'uf',
    'municipio',
    'secao',
    'subclasse',
    'saldomovimentacao',
    'cbo2002ocupacao',
    'categoria',
    'graudeinstrucao',
    'idade',
    'horascontratuais',
    'racacor',
    'sexo',
    'tipoempregador',
    'tipoestabelecimento',
    'tipomovimentacao',
    'tipodedeficiencia',
    'indtrabintermitente',
    'indtrabparcial',
    'salario',
    'tamestabjan',
    'indicadoraprendiz',
    'origemdainformacao',
    'competenciadec',
    'competenciaexc',
    'indicadordeexclusao',
    'indicadordeforadoprazo',
    'unidadesalariocodigo',
    'valorsalariofixo']

# Tabelas CAGED MOVIMENTAÇÃO E CAGED FORA DO PRAZO
colnames2=[
    'competenciamov',
    'regiao',
    'uf',
    'municipio',
    'secao',
    'subclasse',
    'saldomovimentacao',
    'cbo2002ocupacao',
    'categoria',
    'graudeinstrucao',
    'idade',
    'horascontratuais',
    'racacor',
    'sexo',
    'tipoempregador',
    'tipoestabelecimento',
    'tipomovimentacao',
    'tipodedeficiencia',
    'indtrabintermitente',
    'indtrabparcial',
    'salario',
    'tamestabjan',
    'indicadoraprendiz',
    'origemdainformacao',
    'competenciadec',
    'indicadordeforadoprazo',
    'unidadesalariocodigo',
    'valorsalariofixo']

In [5]:
# Verificando se o servidor FTP está ativo
server = "ftp.mtps.gov.br"

try:
    # Conecta ao servidor FTP
    ftp = FTP(server)
    # Faz login (opcional)
    # ftp.login("usuario", "senha")
    # Verifica se a conexão foi bem sucedida
    if ftp.getwelcome():
        print("Conexão bem sucedida ao servidor FTP:", server)
    # Fecha a conexão com o servidor FTP
    ftp.quit()

except Exception as e:
    print("Falha na conexão com o servidor FTP:", server)
    print(e)

Conexão bem sucedida ao servidor FTP: ftp.mtps.gov.br


In [ ]:
ftp = FTP('ftp.mtps.gov.br')
ftp.login()
ftp.cwd('/pdet/microdados/NOVO CAGED')

anos = ['2020', '2021', '2022']
meses = ['01','02','03','04','05','06', '07', '08', '09', '10', '11', '12']

# define o diretório local onde os arquivos serão salvos
local_dir = os.path.join(os.getcwd(), "dados")

if not os.path.exists(local_dir):
    os.makedirs(local_dir)

for ano in anos:
    for mes in meses:
        dir_name = f"{ano}/{ano}{mes}"
        ftp.cwd(f'/pdet/microdados/NOVO CAGED/{dir_name}')
        
        print(f"Baixando arquivos de {mes}/{ano}...")        
        for filename in ftp.nlst():
            if filename.endswith('.7z'):
                local_filename = os.path.join(local_dir, filename)
                with open(local_filename, 'wb') as f:
                    ftp.retrbinary('RETR ' + filename, f.write)
                print(f"Arquivo {filename} baixado com sucesso!")
                with py7zr.SevenZipFile(local_filename, mode='r') as z:
                    z.extractall(path=os.path.dirname(local_filename))
                os.remove(local_filename)
                print(f"Arquivo {filename} descompactado e removido com sucesso!")
                
                new_file = "CAGEDEXC"
                if os.path.isfile(f'{local_dir}/{new_file}{ano}{mes}.txt'):
                    load_caged_to_db(table_name='tb_caged_exc', 
                                     colnames=colnames1, 
                                     origem='CAGEDEXC', 
                                     ano=f'{ano}', 
                                     mes=f'{mes}')
                
                new_file = "CAGEDFOR"
                if os.path.isfile(f'{local_dir}/{new_file}{ano}{mes}.txt'):    
                    load_caged_to_db(table_name='tb_caged_for', 
                                     colnames=colnames2, 
                                     origem='CAGEDFOR', 
                                     ano=f'{ano}', 
                                     mes=f'{mes}')
                    
                new_file = "CAGEDMOV"
                if os.path.isfile(f'{local_dir}/{new_file}{ano}{mes}.txt'):            
                    load_caged_to_db(table_name='tb_caged_mov', 
                                     colnames=colnames2, 
                                     origem='CAGEDMOV', 
                                     ano=f'{ano}', 
                                     mes=f'{mes}')
                    
        ftp.cwd('..')
        
        for arquivo in os.listdir(local_dir):
            caminho = os.path.join(local_dir, arquivo)
            if os.path.isfile(caminho):
                os.remove(caminho)
                
ftp.quit()

In [ ]:
#########################################################################
# TRECHO DE CÓDIGO APENAS PARA O DOWNLOAD E DESCOMPACTAÇÃO DOS ARQUIVOS #
#########################################################################

ftp = FTP('ftp.mtps.gov.br')
ftp.login()
ftp.cwd('/pdet/microdados/NOVO CAGED')

#anos = ['2020', '2021', '2022']
anos = ['2023']
#meses = ['01','02','03','04','05','06', '07', '08', '09', '10', '11', '12']
meses = ['01','02','03','04']


# define o diretório local onde os arquivos serão salvos
local_dir = os.path.join(os.getcwd(), "dados")

if not os.path.exists(local_dir):
    os.makedirs(local_dir)

for ano in anos:
    for mes in meses:
        dir_name = f"{ano}/{ano}{mes}"
        ftp.cwd(f'/pdet/microdados/NOVO CAGED/{dir_name}')
        
        print(f"Baixando arquivos de {mes}/{ano}...")        
        for filename in ftp.nlst():
            if filename.endswith('.7z'):
                local_filename = os.path.join(local_dir, filename)
                with open(local_filename, 'wb') as f:
                    ftp.retrbinary('RETR ' + filename, f.write)
                print(f"Arquivo {filename} baixado com sucesso!")
                with py7zr.SevenZipFile(local_filename, mode='r') as z:
                    z.extractall(path=os.path.dirname(local_filename))
                os.remove(local_filename)
                print(f"Arquivo {filename} descompactado e removido com sucesso!")

In [9]:
# Juntando os arquivos baixados
path_files = 'C:/Users/thiago.silva/TCC/CAGED/dados'

In [4]:
# Juntando os arquivos baixados
path_files = 'C:/Users/thiago.silva/TCC/CAGED/dados'

files_EXC = []
for nome_arquivo in os.listdir(path_files):
    if nome_arquivo.startswith("CAGEDEXC") and nome_arquivo.endswith(".txt"):
        files_EXC.append(nome_arquivo)
        
mes_ano_EXC = []
for nome_arquivo in files_EXC:
    numero = re.search(r"\D*(\d+)", nome_arquivo).group(1)
    mes_ano_EXC.append(numero)
        
files_FOR = []
for nome_arquivo in os.listdir(path_files):
    if nome_arquivo.startswith("CAGEDFOR") and nome_arquivo.endswith(".txt"):
        files_FOR.append(nome_arquivo)
        
mes_ano_FOR = []
for nome_arquivo in files_FOR:
    numero = re.search(r"\D*(\d+)", nome_arquivo).group(1)
    mes_ano_FOR.append(numero)

files_MOV = []
for nome_arquivo in os.listdir(path_files):
    if nome_arquivo.startswith("CAGEDMOV") and nome_arquivo.endswith(".txt"):
        files_MOV.append(nome_arquivo)
        
mes_ano_MOV = []
for nome_arquivo in files_MOV:
    numero = re.search(r"\D*(\d+)", nome_arquivo).group(1)
    mes_ano_MOV.append(numero)

In [5]:
def ajustar_arquivos(lista_arquivos, lista_ano_mes, diretorio):
    for arquivo in lista_arquivos:
        # Caminho completo do arquivo
        caminho_arquivo = os.path.join(diretorio, arquivo)
        
        # Ler o arquivo TXT
        df = pd.read_csv(caminho_arquivo, sep=';')
        
        # Adicionar coluna "ano_mes" e preencher com o elemento correspondente do arquivo de lista_arquivos
        df['ano_mes'] = lista_ano_mes[lista_arquivos.index(arquivo)]
        
        # Converter e salvar o arquivo como CSV
        novo_nome_arquivo = arquivo.replace('.txt', '.csv')
        caminho_novo_arquivo = os.path.join(diretorio, novo_nome_arquivo)
        df.to_csv(caminho_novo_arquivo, sep=';', index=False)
        
        # Deletar o arquivo TXT antigo
        os.remove(caminho_arquivo)

In [6]:
# Ajuste arquivos CAGEDEXC
ajustar_arquivos(files_EXC, mes_ano_EXC, path_files)

# Ajuste arquivos CAGEDFOR
ajustar_arquivos(files_FOR, mes_ano_FOR, path_files)

# Ajuste arquivos CAGEDMOV
ajustar_arquivos(files_MOV, mes_ano_MOV, path_files)